In [5]:
import rdflib

# Querying the Knowledge Graph

Testing the KG and ontology with SPARQL built from the requirements and competency questions.


|Identifier|Requirement or Competency Question                                                                                           |
|---------|-----------------------------------------------------------------------------------------------------------------------------|
|R1           |Articles have title, year, journal, issue, abstract, volume and author                                                       |
|R2           |Each article may refer to one or more experiments                                                                            |
|R3           |Each experiment must have a catalyst                                                                                         |
|R4           |Each experiment may have additional inputs apart from the catalyst                                                           |
|R5           |Each experiment must specify surface area and band gap                                                                       |
|R6           |An experiment must indicate operation conditions, including temperature, pressure, reaction time, medium and operation mode  |
|R7           |Experiments may also indicate type of reactor                                                                 |
|R8           |Electrocatalysis and Photoelectrocatalysis experiments may also indicate electrochemical configuration                       |
|R9           |Photocatalysis and Photoelectrocatalysis experiments must indicate the light source used, including type of light, lamp, wavelength, irradiance and power|
|R10         |TiO2 catalysts must indicate crystal structure                                                                               |
|R11          |Medium reaction must be either gas or liquid                                                                                 |
|R12         |When reaction medium is liquid, the pH value must be indicated                                                               |
|R13         |Operation modes must be either batch or continuous                                                                           |
|R14         |When the operation mode is continuous, the spacial speed value must be indicated                                             |
|R15         |Each experiment may have one or more outputs                                                                                 |
|R16         |When the light source is not solar or solar simulation, the wavelength must be indicated                                     |
|R17         |An experiment may provide conversion metrics, quantum efficiency metrics and electrochemical metrics                         |
|CQ1        |How many experiments are reported per year?                                                                                  |
|CQ2       |How many experiments are reported per country?                                                                               |
|CQ3       |How many experiments are there per type of catalyst?                                                                         |
|CQ4       |Which articles have been published in the ACS nano Journal in 2020 in volume 14?                                            |
|CQ5       |Which experiments use TiO2 as catalyst, liquid medium an produce H2 as output?                                               |
|CQ6       |Which experiments use TiO2 as catalyst, liquid medium and produce the most H2 output in µmol/gh?                             |


In [6]:
g = rdflib.Graph()
g.parse("../ontology/solarchem-core.owl", format='turtle')
g.parse("result/solarchem-kg.nt")
print(len(g))

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#float, Converter=<class 'float'>
Traceback (most recent call last):
  File "/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
ValueError: could not convert string to float: '4 ,6 '
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#float, Converter=<class 'float'>
Traceback (most recent call last):
  File "/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
ValueError: could not convert string to float: '0.26:1'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#float, Converter=<class 'float'>
Traceback (most recent call last):
  File "/Users/aiglesias/miniconda3/envs/kgc/lib/p

2193658


## R1: Articles have title, year, journal, issue, abstract, volume and author

In [4]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX schema: <https://schema.org/>

    SELECT (COUNT (DISTINCT ?article) AS ?count)
    WHERE {
        ?article a schema:ScholarlyArticle ;
            bibo:doi ?doi ;
            schema:name ?name ;
            schema:volumeNumber ?volume ;
            schema:issueNumber ?issue ;
            schema:datePublished ?date ;
            schema:pageStart ?pagestart;
            schema:pageEnd ?pageend;
            schema:isPartOf ?journal .
    } 
""")

for row in q_res:
    print(f"Number of articles with all properties: {row['count'].value}")

Number of articles with all properties: 1254


## R2: Each article may refer to one or more experiments

In [10]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>

    SELECT (COUNT (DISTINCT ?article) AS ?count)
    WHERE {
        ?article a schema:ScholarlyArticle ;
            solar:hasExperimentExecution ?expexec .
    }

""")

for row in q_res:
    print(f"Number of articles with experiments associated: {row['count'].value}")

Number of articles with experiments associated: 1093


## R3: Each experiment must have a catalyst

In [11]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX solarpc: <httsp://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT ?expexec  ?catalyst ?chem
    WHERE {
        ?expexec prov:used ?input .
        ?input a solar:Input ;
            rdfs:label ?catalyst ;
            solar:hasRole solar:Catalyst ;
            solar:hasChemical ?chem
    } LIMIT 10

""")

for row in q_res:
    print(f"{row.expexec} {row.chem} {row.catalyst}")

http://w3id.org/solar/i/ExpExec/7837 http://purl.obolibrary.org/obo/CHEBI_50860 Reduced graphene Oxide
http://w3id.org/solar/i/ExpExec/5111 http://www.wikidata.org/entity/Q193521 TiO2
http://w3id.org/solar/i/ExpExec/6617 http://w3id.org/solar/i/Chemical/CsPbBr3 CsPbBr3
http://w3id.org/solar/i/ExpExec/7686 http://w3id.org/solar/i/Chemical/TiMOF+and+ZnTCPP TiMOF and ZnTCPP
http://w3id.org/solar/i/ExpExec/4715 http://www.wikidata.org/entity/Q193521 TiO2
http://w3id.org/solar/i/ExpExec/7262 http://w3id.org/solar/i/Chemical/g-C3N4 g-C3N4
http://w3id.org/solar/i/ExpExec/3190 http://purl.obolibrary.org/obo/CHEBI_81908 Cu2O
http://w3id.org/solar/i/ExpExec/4840 http://w3id.org/solar/i/Chemical/Phthalocyanine+%28Co%29 Phthalocyanine (Co)
http://w3id.org/solar/i/ExpExec/5032 http://w3id.org/solar/i/Chemical/Al2O3 Al2O3
http://w3id.org/solar/i/ExpExec/5766 http://www.wikidata.org/entity/Q193521 TiO2


## R4: Each experiment may have additional inputs apart from the catalyst

In [12]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX solarpc: <httsp://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT DISTINCT  ?expexec ?inputlabel ?role
    WHERE {
        ?expexec prov:used ?input .
        ?input a solar:Input ;
            rdfs:label ?inputlabel ;
            solar:hasRole ?role .
        FILTER(?role != solar:Catalyst)
    } LIMIT 10

""")

for row in q_res:
    print(f"{row.expexec} {row.inputlabel} {row.role}")

http://w3id.org/solar/i/ExpExec/5460 H2O https://w3id.org/solar/o/pc#Reductant
http://w3id.org/solar/i/ExpExec/7565 Bi2WO6 https://w3id.org/solar/o/core#Co-catalyst
http://w3id.org/solar/i/ExpExec/4948 Pt https://w3id.org/solar/o/core#Co-catalyst
http://w3id.org/solar/i/ExpExec/1929 CdS https://w3id.org/solar/o/core#Co-catalyst
http://w3id.org/solar/i/ExpExec/4766 SiO2 https://w3id.org/solar/o/pc#Support
http://w3id.org/solar/i/ExpExec/6801 Co(NO3)2⋅6H2O https://w3id.org/solar/o/core#Co-catalyst
http://w3id.org/solar/i/ExpExec/3760 CuAlGaO4 https://w3id.org/solar/o/core#Co-catalyst
http://w3id.org/solar/i/ExpExec/7193 H2O https://w3id.org/solar/o/pc#Reductant
http://w3id.org/solar/i/ExpExec/5838 H2O https://w3id.org/solar/o/pc#Reductant
http://w3id.org/solar/i/ExpExec/839 CH3OH https://w3id.org/solar/o/pc#Reductant


## R5: Each experiment must specify surface area and band gap

In [13]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX solarpc: <httsp://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>

    SELECT DISTINCT ?expexec ?surfacearea ?saunit  ?bgap ?bgapunit
    WHERE {
        ?expexec prov:used ?input .
        ?input a solar:Input ;
            solar:hasBandGap [ qudt:numericValue ?bgap; qudt:unit ?bgapunit ] ;
            solar:hasSurfaceArea [ qudt:numericValue ?surfacearea; qudt:unit ?saunit ] .
    } LIMIT 10

""")

for row in q_res:
    print(f"{row.expexec} Bandgap: {row.bgap} {row.bgapunit} | Surface area: {row.surfacearea} {row.saunit}")
    #print(f"{row.expexec}  | Surface area: {row.surfacearea} {row.saunit}")

http://w3id.org/solar/i/ExpExec/5111 Bandgap: 3.2 http://qudt.org/vocab/unit/EV | Surface area: 48.0 http://qudt.org/vocab/unit/M2-PER-GM
http://w3id.org/solar/i/ExpExec/6388 Bandgap: 1.3 http://qudt.org/vocab/unit/EV | Surface area: 92.12 http://qudt.org/vocab/unit/M2-PER-GM
http://w3id.org/solar/i/ExpExec/4765 Bandgap: 2.92 http://qudt.org/vocab/unit/EV | Surface area: 148.0 http://qudt.org/vocab/unit/M2-PER-GM
http://w3id.org/solar/i/ExpExec/3662 Bandgap: 3.16 http://qudt.org/vocab/unit/EV | Surface area: 48.0 http://qudt.org/vocab/unit/M2-PER-GM
http://w3id.org/solar/i/ExpExec/6571 Bandgap: 2.34 http://qudt.org/vocab/unit/EV | Surface area: 39.12 http://qudt.org/vocab/unit/M2-PER-GM
http://w3id.org/solar/i/ExpExec/2222 Bandgap: 3.4 http://qudt.org/vocab/unit/EV | Surface area: 240.0 http://qudt.org/vocab/unit/M2-PER-GM
http://w3id.org/solar/i/ExpExec/1702 Bandgap: 2.85 http://qudt.org/vocab/unit/EV | Surface area: 52.4 http://qudt.org/vocab/unit/M2-PER-GM
http://w3id.org/solar/i/Ex

## R6: An experiment must indicate operation conditions, including temperature, pressure, reaction time, medium and operation mode
Note: Medium and operation mode classes in onto, not in this KG's version

In [14]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX solarpc: <httsp://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>

    SELECT (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?expexec solar:hasCondition ?condition .
        ?condition a ?condition_type .
        VALUES ?condition_type {solar:TimeCondition solar:PressureCondition solar:TemperatureCondition}
    }
""")

for row in q_res:
    print(f"Number of experiments with conditions: {row['count'].value}")

Number of experiments with conditions: 6664


## R7: Experiments may also indicate type of reactor

In [21]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX solarpc: <https://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>

    SELECT (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?expexec solar:hasCondition ?reactorCondition .
        ?reactorCondition a solar:ReactorCondition
    }
""")

for row in q_res:
   print(f"Number of articles with reactor conditions: {row['count'].value}")

Number of articles with reactor conditions: 6663


## R8: Electrocatalysis and Photoelectrocatalysis experiments may also indicate electrochemical configuration
No experiments so far with this condition, only from photocatalysis

In [22]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX solarpc: <httsp://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>

    SELECT (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?expexec solar:hasCondition ?condition .
        ?condition a solar:ElectrochemicalConfiguration .
    }
""")

for row in q_res:
    print(f"Number of articles with electrochemical configuration: {row['count'].value}")

Number of articles with electrochemical configuration: 0


## R9: Photocatalysis and Photoelectrocatalysis experiments must indicate the light source used, including type of light, lamp, wavelength, irradiance, illuminated area and power

In [23]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX solarpc: <httsp://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>
    PREFIX obi: <http://purl.obolibrary.org/obo/OBI_>

    SELECT DISTINCT ?expexec ?lamp ?wl ?irradiance ?power ?illum_area
    WHERE {
        ?expexec solar:hasLightSource ?light_source.
        ?light_source a obi:0400065 ;
            solar:hasLamp ?lamp ;
            solar:hasWavelength ?wl ;
            solar:hasIrradiance [qudt:numericValue ?irradiance] ;
            solar:hasPower [qudt:numericValue ?power] ;
            solar:hasIlluminatedArea [qudt:numericValue ?illum_area] .
            
    } LIMIT 5
""")

for row in q_res:
    print(f"{row.expexec}\n\tLamp: {row.lamp}\n\tWavelength: {row.wl}\n\tIrradiance: {row.irradiance}\n\tPower: {row.power}\n\tIlluminated Area: {row.illum_area}")

https://w3id.org/solar/i/ExpExec/3734
	Lamp: https://w3id.org/solar/o/core#XenonLamp
	Wavelength: https://w3id.org/solar/o/core#192-280WL
	Irradiance: 2700.0
	Power: 300.0
	Illuminated Area: 368.0
https://w3id.org/solar/i/ExpExec/3022
	Lamp: https://w3id.org/solar/o/core#XenonLamp
	Wavelength: https://w3id.org/solar/o/core#400-780WL
	Irradiance: 350.0
	Power: 500.0
	Illuminated Area: 238.0
https://w3id.org/solar/i/ExpExec/3008
	Lamp: https://w3id.org/solar/o/core#Mercury-XenonLamp
	Wavelength: https://w3id.org/solar/o/core#192-280WL
	Irradiance: 200.0
	Power: 254.0
	Illuminated Area: 196.0
https://w3id.org/solar/i/ExpExec/3059
	Lamp: https://w3id.org/solar/o/core#XenonLamp
	Wavelength: https://w3id.org/solar/o/core#400-780WL
	Irradiance: 350.0
	Power: 500.0
	Illuminated Area: 238.0
https://w3id.org/solar/i/ExpExec/6682
	Lamp: https://w3id.org/solar/o/core#MercuryLamp
	Wavelength: https://w3id.org/solar/o/core#192-280WL
	Irradiance: 20.0
	Power: 350.0
	Illuminated Area: 0.01


## R10: TiO2 catalysts must indicate crystal structure

In [24]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX solarpc: <httsp://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>

    SELECT ?input ?crystal_str
    WHERE {
        ?input a solar:Input ;
            rdfs:label "TiO2" ;
            solar:hasRole solar:Catalyst ;
            solar:crystalStructure ?crystal_str .
    } LIMIT 10

""")

for row in q_res:
    print(f"{row.input}  {row.crystal_str}")

https://w3id.org/solar/i/Input/Catalyst/7704-TiO2  Anatase
https://w3id.org/solar/i/Input/Catalyst/2426-TiO2  Anatase
https://w3id.org/solar/i/Input/Catalyst/2296-TiO2  Anatase
https://w3id.org/solar/i/Input/Catalyst/5831-TiO2  Mix
https://w3id.org/solar/i/Input/Catalyst/7771-TiO2  Mix
https://w3id.org/solar/i/Input/Catalyst/785-TiO2  Anatase
https://w3id.org/solar/i/Input/Catalyst/285-TiO2  Mix
https://w3id.org/solar/i/Input/Catalyst/7814-TiO2  Anatase
https://w3id.org/solar/i/Input/Catalyst/21-TiO2  P25
https://w3id.org/solar/i/Input/Catalyst/4652-TiO2  Rutile


## R11: Medium reaction must be either gas or liquid

In [25]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX solarpc: <httsp://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>

    SELECT DISTINCT ?label
    WHERE {
        ?medium a solar:ReactionMediumCondition ;
                rdfs:label ?label
    }
""")

for row in q_res:
    print(f"{row.label}")

Gas Medium
Liquid Medium


## R12: When reaction medium is liquid, the pH value must be indicated

In [26]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX solarpc: <httsp://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>

    SELECT ?expexec ?ph
    WHERE {
        ?expexec solar:hasCondition solar:LiquidMedium ;
            solar:hasCondition ?ph_cond .
        ?ph_cond a solar:pHCondition ;
            qudt:numericValue ?ph .
    } LIMIT 10
""")

for row in q_res:
    print(f"{row.expexec} pH: {row.ph}")

https://w3id.org/solar/i/ExpExec/6197 pH: 7.0
https://w3id.org/solar/i/ExpExec/5059 pH: 7.0
https://w3id.org/solar/i/ExpExec/6293 pH: 7.0
https://w3id.org/solar/i/ExpExec/3299 pH: 8.0
https://w3id.org/solar/i/ExpExec/7792 pH: 3.0
https://w3id.org/solar/i/ExpExec/6660 pH: 7.0
https://w3id.org/solar/i/ExpExec/6300 pH: 7.0
https://w3id.org/solar/i/ExpExec/4817 pH: 7.0
https://w3id.org/solar/i/ExpExec/5205 pH: 7.0
https://w3id.org/solar/i/ExpExec/5216 pH: 7.0


## R13: Operation modes must be either batch or continuous

In [27]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX solarpc: <httsp://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>

    SELECT DISTINCT ?label
    WHERE {
        ?medium a solar:OperationModeCondition ;
                rdfs:label ?label
    }
""")

for row in q_res:
    print(f"{row.label}")

Batch Mode
Continuous Mode


## R14: When the operation mode is continuous, the spacial speed value must be indicated


In [28]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX solarpc: <httsp://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>

    SELECT ?expexec ?restime ?unit
    WHERE {
        ?expexec solar:hasCondition solar:ContinuousMode ;
            solar:hasCondition ?restime_cond .
        ?restime_cond a solar:ResidenceTimeCondition ;
            qudt:numericValue ?restime;
            qudt:unit ?unit.
    } LIMIT 5
""")

for row in q_res:
    print(f"{row.expexec} Residence time: {row.restime} {row.unit}")

https://w3id.org/solar/i/ExpExec/4914 Residence time: 0.24 http://qudt.org/vocab/unit/PER-MIN
https://w3id.org/solar/i/ExpExec/34 Residence time: 83.3 http://qudt.org/vocab/unit/PER-MIN
https://w3id.org/solar/i/ExpExec/4897 Residence time: 0.03 http://qudt.org/vocab/unit/PER-MIN
https://w3id.org/solar/i/ExpExec/687 Residence time: 15.0 http://qudt.org/vocab/unit/PER-MIN
https://w3id.org/solar/i/ExpExec/2974 Residence time: 360.0 http://qudt.org/vocab/unit/PER-MIN


## R15: Each experiment may have one or more outputs

In [35]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX solarpc: <https://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>
    PREFIX prov: <http://www.w3.org/ns/prov#>

    SELECT (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?output a solar:Output ;
            prov:wasGeneratedBy ?expexec .
    }

""")

for row in q_res:
    print(f"Number of experiments with outputs: {row['count'].value}")

Number of experiments with outputs: 6664


## R16: When the light source is not solar or solar simulation, the wavelength must be indicated

In [36]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX solarpc: <httsp://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>
    PREFIX obi: <http://purl.obolibrary.org/obo/OBI_>

    SELECT (COUNT (DISTINCT ?light_source) AS ?count)
    WHERE {
        ?light_source a ?type ;
            solar:hasWavelength ?wl .
        FILTER (?type != solar:SolarLightSource || ?type != solar:SolarSimulatorLightSource )
    } #LIMIT 10
""")

for row in q_res:
    print(f"Number of lightsources with all properties: {row['count'].value}")

Number of lightsources with all properties: 5398


## R17: An experiment may provide conversion metrics, quantum efficiency metrics and electrochemical metrics
Not yet in the KG, only photocatalysis experiments

## CQ1: How many experiments are reported per year?

In [38]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX schema: <https://schema.org/>

    SELECT ?year (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?article a schema:ScholarlyArticle ;
            schema:datePublished ?year ;
            solar:hasExperimentExecution ?expexec
    } GROUP BY ?year
    ORDER BY DESC (?year)

""")

for row in q_res:
    print(f"Experiments in {row.year}: {row['count'].value}")

Experiments in 2022: 263
Experiments in 2021: 1176
Experiments in 2020: 1149
Experiments in 2019: 311
Experiments in 2018: 530
Experiments in 2017: 719
Experiments in 2016: 343
Experiments in 2015: 579
Experiments in 2014: 392
Experiments in 2013: 347
Experiments in 2012: 202
Experiments in 2011: 201
Experiments in 2010: 41
Experiments in 2009: 41
Experiments in 2008: 46
Experiments in 2007: 53
Experiments in 2006: 15
Experiments in 2005: 20
Experiments in 2004: 19
Experiments in 2003: 22
Experiments in 2002: 18
Experiments in 2001: 19
Experiments in 2000: 4
Experiments in 1999: 8
Experiments in 1998: 21
Experiments in 1997: 30
Experiments in 1996: 2
Experiments in 1995: 17
Experiments in 1994: 20
Experiments in 1993: 14
Experiments in 1992: 9
Experiments in 1991: 1
Experiments in 1990: 2
Experiments in 1987: 1
Experiments in 1984: 6
Experiments in 1979: 7


## CQ2: How many experiments are reported per country?


In [40]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX schema: <https://schema.org/>

    SELECT ?country (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?article a schema:ScholarlyArticle ;
                solar:hasExperimentExecution ?expexec ;
                schema:author ?author .
        ?author schema:affiliation ?affiliation .
        ?affiliation schema:addressCountry ?country
    } GROUP BY ?country
    ORDER BY DESC (?count)

""")

for row in q_res:
    print(f"Experiments in {row.country}: {row['count'].value}")

Experiments in https://w3id.org/solar/i/Country/CN: 6086
Experiments in https://w3id.org/solar/i/Country/JP: 5051
Experiments in https://w3id.org/solar/i/Country/ES: 4973
Experiments in https://w3id.org/solar/i/Country/AU: 4970
Experiments in https://w3id.org/solar/i/Country/SA: 4947
Experiments in https://w3id.org/solar/i/Country/US: 4945
Experiments in https://w3id.org/solar/i/Country/MY: 4933
Experiments in https://w3id.org/solar/i/Country/SG: 4903
Experiments in https://w3id.org/solar/i/Country/DE: 4870
Experiments in https://w3id.org/solar/i/Country/PK: 4862
Experiments in https://w3id.org/solar/i/Country/GB: 4838
Experiments in https://w3id.org/solar/i/Country/HK: 4831
Experiments in https://w3id.org/solar/i/Country/TW: 4821
Experiments in https://w3id.org/solar/i/Country/IR: 4817
Experiments in https://w3id.org/solar/i/Country/NL: 4807
Experiments in https://w3id.org/solar/i/Country/KR: 4806
Experiments in https://w3id.org/solar/i/Country/IN: 4804
Experiments in https://w3id.org

## CQ3: How many experiments are there per type of catalyst?

In [41]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>

    SELECT ?catalyst (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?expexec prov:used ?input .
        ?input solar:hasRole solar:Catalyst ;
            rdfs:label ?catalyst .
    } GROUP BY ?catalyst
    ORDER BY DESC (?count)

""")

for row in q_res:
    print(f"Experiments with {row.catalyst}: {row['count'].value}")

Experiments with TiO2: 2131
Experiments with g-C3N4: 745
Experiments with Ti-KIT-6: 201
Experiments with ZnO: 182
Experiments with CeO2: 160
Experiments with ZnS: 138
Experiments with CdS: 112
Experiments with BiOCl: 85
Experiments with Cu2O: 83
Experiments with Bi2WO6: 82
Experiments with CsPbBr3: 66
Experiments with WO3: 65
Experiments with BiVO4: 64
Experiments with LDH: 55
Experiments with MgIn2S4: 55
Experiments with Ga2O3: 54
Experiments with In2O3: 41
Experiments with CuAlGaO4: 40
Experiments with AgBr: 40
Experiments with K2Ti6O13: 38
Experiments with ZIF-9: 38
Experiments with ZrO2: 38
Experiments with Zn2GeO4: 34
Experiments with Pd(bpy)-F23: 34
Experiments with InTaO4: 31
Experiments with UIO-66-NH2(Zr): 31
Experiments with BiOBr: 30
Experiments with Bi3O4Cl: 30
Experiments with SrTiO3: 29
Experiments with MoS2: 29
Experiments with MoO3-x: 28
Experiments with Carbon Nitride: 26
Experiments with C3N4: 26
Experiments with ZnIn2S4: 25
Experiments with [Zn1.5Cu1.5Ga(OH)8]2(CO3)2

## CQ4: Which articles have been published in the ACS nano Journal in 2020 in volume 14?

In [27]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX schema: <https://schema.org/>

    SELECT ?article ?title ?doi
    WHERE {
        ?article a schema:ScholarlyArticle ;
            schema:name ?title ;
            bibo:doi ?doi ;
            schema:volumeNumber "14" ;
            schema:datePublished "2020"^^xsd:gYear ;
            schema:isPartOf "ACS nano" .
    } 
""")

for row in q_res:
    print(f"{row.article} {row.title} {row.doi}")

https://w3id.org/solar/i/Article/1031 Rare-Earth Single-Atom La–N Charge-Transfer Bridge on Carbon Nitride for Highly Efficient and Selective Photocatalytic CO<sub>2</sub> Reduction https://doi.org/10.1021/acsnano.0c07083
https://w3id.org/solar/i/Article/894 Room-Temperature Synthesis of Single Iron Site by Electrofiltration for Photoreduction of CO<sub>2</sub> into Tunable Syngas https://doi.org/10.1021/acsnano.0c02162
https://w3id.org/solar/i/Article/1012 Identification of Halogen-Associated Active Sites on Bismuth-Based Perovskite Quantum Dots for Efficient and Selective CO<sub>2</sub>-to-CO Photoreduction https://doi.org/10.1021/acsnano.0c04659
https://w3id.org/solar/i/Article/944 A Single Cu-Center Containing Enzyme-Mimic Enabling Full Photosynthesis under CO<sub>2</sub> Reduction https://doi.org/10.1021/acsnano.0c02940
https://w3id.org/solar/i/Article/911 Direct Z-Scheme Heterojunction of Semicoherent FAPbBr<sub>3</sub>/Bi<sub>2</sub>WO<sub>6</sub> Interface for Photoredox Reacti

## CQ5: Which experiments use TiO2 as catalyst, liquid medium an produce H2 as output?
Slow with RDFLib, works fine in GraphDB

In [ ]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>

    SELECT ?expexec
    WHERE {
        ?expexec prov:used ?input ;
            solar:hasCondition solar:LiquidMedium .
            
        ?input solar:hasRole solar:Catalyst ;
            rdfs:label "TiO2" .
    
        ?output rdfs:label "H2" ;
            prov:wasGeneratedBy ?expexec .
    }
""")

for row in q_res:
    print(f"Experiments: {row['count'].value}")

## CQ6: Which experiments use TiO2 as catalyst, liquid medium and produce the most H2 output in µmol/gh?

In [ ]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX prov: <http://www.w3.org/ns/prov#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX unit: <http://qudt.org/vocab/unit/>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>

    SELECT ?expexec ?value
        WHERE {
            ?expexec prov:used ?input ;
                solar:hasCondition solar:LiquidMedium .
                
            ?input solar:hasRole solar:Catalyst ;
                rdfs:label "TiO2" .
        
            ?output rdfs:label "H2" ;
                qudt:unit unit:MicroMOL-PER-GM-HR ;
                prov:wasGeneratedBy ?expexec ;
                qudt:numericValue ?value .
    } ORDER BY DESC(?value)
""")

for row in q_res:
    print(f"Experiments: {row['count'].value}")